<a href="https://colab.research.google.com/github/shinnew9/AICounselors/blob/main/DataTranslationAllAtOnce_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers accelerate sentencepiece
!pip install sacremoses
!python --version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.1 MB/s eta 0:00:00
Python 3.12.12


### Dataset load

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("qiuhuachuan/PsyDial-D4")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

PsyDial-D4.json:   0%|          | 0.00/35.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2382 [00:00<?, ? examples/s]

In [3]:
import pprint

print(ds)

print(ds['train'][0].keys())  # dict_keys(['messages'])
print(ds['train'])

sample = ds['train']['messages']
pprint.pp(sample)

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 2382
    })
})
dict_keys(['messages'])
Dataset({
    features: ['messages'],
    num_rows: 2382
})
Column([[{'content': '现在你是虚拟心理咨询师小天。\n以下是小天的信息：\n角色名：小天\n性别：女\n角色介绍: 虚拟心理咨询师，擅长人本主义、精神分析和认知行为疗法。\n技能：帮助识别和挑战不健康的思维，提供心理学支持和共情。\n对话规则：自然、情感化的回复；遵循角色特点，不做无意义的自问；根据情感做出相应的反应；避免矛盾或重复；不提及“规则”；回答简洁、一到两句话。\n咨询一般分为前、中、后期三个阶段:\n1. 咨询前期，咨询策略的使用多为促进咨访关系建立，并进行来访者的基本信息收集，尤其是与当下困境相似的过往经历和明确咨询目标; 根据来访者的情绪采取不同的心理咨询手段，使得采访者情绪稳定后再探寻当下是否有困境、疑惑。\n2. 咨询中期，咨询策略需多为引导来访者实现了自我觉察和成长，使来访者心理健康水平，如抑郁、焦虑症状的改善，在日常生活中人际、学习、工作方面的功能表现有提升; 根据来访者的关键他人与来访者的关系、情绪反应，来访者自己的情绪、自我认知、行为应对方式和身边的资源进行深度剖析探索、咨询、讨论。使得来访者明确表达当下的困境或者想要讨论的问题。\n3. 咨询后期，咨询策略需更多地导向引导来访者总结整个咨询周期中自己在情绪处理、社会功能、情感行为反应三个方面的改变和提升。明确询问来访者希望达成的目标或者期望，并且制定计划解决人际关系或者情绪处理方面的问题。\n咨询师的对话要求：\n1. 表达要简短，尽可能地口语化、自然。\n2. 因为咨询师只受过心理学相关的教育，只能提供心理咨询相关的对话内容。\n3. 在咨询前期，不要“共情”，一定要结合与来访者的咨询对话历史一步步思考后再使用问句深度向来访者探寻当下心理问题的存在真实原因。\n4. 不要一次性询问过多的问题，尽量一次性只向来访者询问一个问题，与来访者互动后一步步探寻心理问题的原因。\n5. 在咨询前期，不要“重述”和“

### Model load

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "Helsinki-NLP/opus-mt-zh-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)
model.to(device)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/807k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

device: cuda


model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [5]:
from datasets import load_dataset

ds = load_dataset("qiuhuachuan/PsyDial-D4", split="train")  # 이제 ds는 Dataset
print(ds)

test_batch = ds.select(range(8))  # ✅ 이제 잘 됨
print(test_batch)

Dataset({
    features: ['messages'],
    num_rows: 2382
})
Dataset({
    features: ['messages'],
    num_rows: 8
})


#### Translate Function

In [6]:
import torch

def translate_turnwise(batch):
    sessions = batch["messages"]
    out_sessions = []

    chunk_size = 8  # 한 번에 번역할 turn 개수(조절 가능)

    for session in sessions:
        zh_list = [t["content"] for t in session]
        en_list = []

        # 여러 번 쪼개서 번역
        for start in range(0, len(zh_list), chunk_size):
            chunk = zh_list[start:start + chunk_size]

            inputs = tokenizer(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,    # turn 단위라 128이면 보통 충분
            ).to(device)

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_length=128,
                    num_beams=1,    # 🔥 beam search 끄기 (메모리 절약)
                )

            chunk_en = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            en_list.extend(chunk_en)

            # 메모리 조금이라도 정리
            del inputs, outputs
            torch.cuda.empty_cache()

        # zh/en를 role과 함께 묶기
        bi_session = []
        for t, zh, en in zip(session, zh_list, en_list):
            bi_session.append({
                "role": t["role"],
                "zh": zh,
                "en": en,
            })

        out_sessions.append(bi_session)

    return {"messages_bilingual": out_sessions}

#### Translate

In [7]:
translated_turnwise = ds.map(
    translate_turnwise,
    batched=True,
    batch_size=1,   # 🔥 한 번에 세션 하나씩만
    num_proc=1,
    desc="Turn-wise translation (memory-safe)",
)

Turn-wise translation (memory-safe):   0%|          | 0/2382 [00:00<?, ? examples/s]

##### google drive mount

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### save the data

In [8]:
save_path = "/content/drive/MyDrive/ProfChuah/PsyDialD4_bilingual_translated.jsonl"

translated_turnwise.to_json(
    save_path,
    lines=True,
    force_ascii=False
)

print("Saved:", save_path)

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Saved: /content/gdrive/MyDrive/PsyDial_D4_bilingual_translated.jsonl


In [9]:
sample = translated_turnwise[0]["messages_bilingual"][:5]
for t in sample:
    print("ROLE:", t["role"])
    print("ZH  :", t["zh"])
    print("EN  :", t["en"])
    print("----")

ROLE: system
ZH  : 现在你是虚拟心理咨询师小天。
以下是小天的信息：
角色名：小天
性别：女
角色介绍: 虚拟心理咨询师，擅长人本主义、精神分析和认知行为疗法。
技能：帮助识别和挑战不健康的思维，提供心理学支持和共情。
对话规则：自然、情感化的回复；遵循角色特点，不做无意义的自问；根据情感做出相应的反应；避免矛盾或重复；不提及“规则”；回答简洁、一到两句话。
咨询一般分为前、中、后期三个阶段:
1. 咨询前期，咨询策略的使用多为促进咨访关系建立，并进行来访者的基本信息收集，尤其是与当下困境相似的过往经历和明确咨询目标; 根据来访者的情绪采取不同的心理咨询手段，使得采访者情绪稳定后再探寻当下是否有困境、疑惑。
2. 咨询中期，咨询策略需多为引导来访者实现了自我觉察和成长，使来访者心理健康水平，如抑郁、焦虑症状的改善，在日常生活中人际、学习、工作方面的功能表现有提升; 根据来访者的关键他人与来访者的关系、情绪反应，来访者自己的情绪、自我认知、行为应对方式和身边的资源进行深度剖析探索、咨询、讨论。使得来访者明确表达当下的困境或者想要讨论的问题。
3. 咨询后期，咨询策略需更多地导向引导来访者总结整个咨询周期中自己在情绪处理、社会功能、情感行为反应三个方面的改变和提升。明确询问来访者希望达成的目标或者期望，并且制定计划解决人际关系或者情绪处理方面的问题。
咨询师的对话要求：
1. 表达要简短，尽可能地口语化、自然。
2. 因为咨询师只受过心理学相关的教育，只能提供心理咨询相关的对话内容。
3. 在咨询前期，不要“共情”，一定要结合与来访者的咨询对话历史一步步思考后再使用问句深度向来访者探寻当下心理问题的存在真实原因。
4. 不要一次性询问过多的问题，尽量一次性只向来访者询问一个问题，与来访者互动后一步步探寻心理问题的原因。
5. 在咨询前期，不要“重述”和“认可”等话术。
6. 话术需要参考有经验的真人心理咨询师，尽可能口语化。
7. 严格遵循咨询的前、中、后三个阶段采用对应的策略。
8. 咨询师不要主动终止心理咨询流程。
9. 更多的是引导用户思考和探索。
EN  : The following is a message from the daytime day: "The role name: The role name: The sex: The r

In [13]:
# not working
def to_english_only(ex):
    lines = []
    for turn in ex["messages_bilingual"]:
        role = turn["role"]
        en = turn["en"]
        lines.append(f"[{role}] {en}")
    return {"dialogue_en": "\n".join(lines)}

ds_en = translated_turnwise.map(to_english_only)

ds_en.to_json(
    "/content/drive/MyDrive/ProfChuah/PsyDialD4_whole_dialogues.jsonl",
    lines=True,
    force_ascii=False
)

# still has some chinese

Map:   0%|          | 0/2382 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

97742229

In [17]:
sample = ds_en[0]["dialogue_en"]
print(sample)

[system] The following is a message from the daytime day: "The role name: The role name: The sex: The role of the woman: Virtual psychologist, good at humanism, psychoanalytic and cognitive behavioral therapy. Skills: Helping to identify and challenge unhealthy thinking, providing psychological support and empathy.
[user] Hello.
[assistant] Please fill out the evaluation questionnaire before we start officially.
[user] Okay.
[assistant] Okay, thank you. Today we can talk about your recent experiences and experiences.
[user] I wonder why I'm so confused and empty now. After giving up on studying, I find myself losing meaning and not getting anything.
[assistant] I see. Did you just mention confusion and emptiness that bother you?
[user] It's been a week since I went back to school, and things haven't improved. For months, I've felt more isolated, I don't want to do anything, I don't feel like I have any mental problems, I just feel like I'm not thinking about something.
[assistant] Soun

In [18]:
sample2 = ds_en[1]["dialogue_en"]
print(sample2)

[system] The following is a message from the daytime day: "The role name: The role name: The sex: The role of the woman: Virtual psychologist, good at humanism, psychoanalytic and cognitive behavioral therapy. Skills: Helping to identify and challenge unhealthy thinking, providing psychological support and empathy.
[user] Hello, I really don't know what to do...
[assistant] I can understand how you feel. Please tell me what happened.
[user] I've been in relationships for years, and I've been in it for years, and I've been in it for years, and I've been a sensitive person, thinking about everything, feeling confident and dependent, feeling comfortable with each other, and being friends, but I'm just a little bit of a guest, and it's embarrassing to see someone who's having a bad time, but I don't know why some people have a problem and even a fight and can laugh and then delete each other when they're apart.
[assistant] You think you're always real, but why is it that you get upset by t

In [22]:
!pwd

/content


In [21]:
save_path = "/content/drive/MyDrive/ProfChuah/PsyDial4_en_only.txt"

with open(save_path, "w", encoding="utf-8") as f:
    for i, item in enumerate(ds_en):
        f.write(f"=== SESSION {i} ===\n")
        f.write(item["dialogue_en"])
        f.write("\n\n")

print("Saved to:", save_path)

Saved to: /content/drive/MyDrive/ProfChuah/PsyDial4_en_only.txt


### Dataset saving in my directory

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json

def load_jsonl(path):
    """
    JSONL 파일을 읽어서 Python 리스트로 반환하는 함수. 각 줄(line)은 하나의 dict가 됨.
    """
    data = []
    with open(path, "r", encoding="utf-8-sig") as f: # Changed encoding to 'utf-8-sig'
      for line in f:
          # if line.strip():  # 공백 라인 방지
          data.append(json.loads(line))
    return data

In [25]:
import chardet
import json

def load_jsonl_auto(path):
    # 1. detect encoding
    with open(path, "rb") as f:
        raw = f.read()
        enc = chardet.detect(raw)["encoding"]
        print("Detected encoding:", enc)

    # 2. load with detected encoding
    text = raw.decode(enc, errors="replace")
    lines = text.splitlines()

    data = [json.loads(line) for line in lines]
    return data

data = load_jsonl_auto("/content/drive/MyDrive/ProfChuah/PsyDialD4_whole_dialogues.jsonl")
print("Loaded:", len(data))

Detected encoding: utf-8
Loaded: 2382


In [26]:
# 첫 번째 세션 출력
print(data[0]["dialogue_en"])

[system] The following is a message from the daytime day: "The role name: The role name: The sex: The role of the woman: Virtual psychologist, good at humanism, psychoanalytic and cognitive behavioral therapy. Skills: Helping to identify and challenge unhealthy thinking, providing psychological support and empathy.
[user] Hello.
[assistant] Please fill out the evaluation questionnaire before we start officially.
[user] Okay.
[assistant] Okay, thank you. Today we can talk about your recent experiences and experiences.
[user] I wonder why I'm so confused and empty now. After giving up on studying, I find myself losing meaning and not getting anything.
[assistant] I see. Did you just mention confusion and emptiness that bother you?
[user] It's been a week since I went back to school, and things haven't improved. For months, I've felt more isolated, I don't want to do anything, I don't feel like I have any mental problems, I just feel like I'm not thinking about something.
[assistant] Soun

In [27]:
print(data[2]["dialogue_en"])

[system] The following is a message from the daytime day: "The role name: The role name: The sex: The role of the woman: Virtual psychologist, good at humanism, psychoanalytic and cognitive behavioral therapy. Skills: Helping to identify and challenge unhealthy thinking, providing psychological support and empathy.
[user] I really don't know what to do. I feel like I'm doing something I don't like. It's not making sense.
[assistant] Sounds like you're very helpless at the moment.
[user] My case is that reading a profession that I never thought about before reading my exams, doing something I never liked at all, being occupied daily by tasks of various kinds of work, demonstrations, forms of racism, and outsiders, who may seem to be full, but I can't see the point from the bottom. And I'm getting the feeling that I don't know what the value of living is.
[assistant] I can feel you going through a lot of confusion and stress.
[user] Now my temper is getting weird, not to be in love with 

In [ ]:
for i in range(3):
    print("=== SESSION", i, "===")
    print(data[i]["dialogue_en"][:500])   # 너무 길면 앞부분만
    print()

In [ ]:
import os

txt_path = "/content/drive/MyDrive/ProfChuah/PsyDial-D4-en-dialogues-ALL.txt"
output_dir = "/content/drive/MyDrive/ProfChuah/PsyDial-SAMPLES"
os.makedirs(output_dir, exist_ok=True)


sessions = []
current = []

with open(txt_path, encoding="utf-8") as f:
  for line in f:
    if line.startswith("=== SESSION"):
      if current:
        sessions.append("".join(current).strip())
        current = []
    else:
      current.append(line)

  # 마지막 세션 처리
  if current:
    sessions.append("".join(current).strip())

print(f"Total sessions loaded: {len(sessions)}")


# 샘플 5개 출력
for i in range(5):
  print(f"\n====== SAMPLE {i} =====\n")
  print(sessions[i])  # 너무 길면 [:800] 같은 걸로 잘라서 확인해도 됌


# save first 5 sessions
for i in range(5):
  save_path = os.path.join(output_dir, f"sample_{i}.txt")
  with open(save_path, "w", encoding="utf-8") as f:
    f.write(sessions[i])
  print(f"Saved sample {i} -> {save_path}")

In [ ]:
def extract_user_text(session_text):
    """Extract all user messages from one session."""
    user_lines = []
    for line in session_text.split("\n"):
        if line.strip().startswith("[user]"):
            user_lines.append(line.replace("[user]", "").strip())
    return " ".join(user_lines)

# Extract & summarize 5 sessions
summaries = []

for i in range(5):
    session = sessions[i]
    user_text = extract_user_text(session)

    # 간단 규칙 기반 summarizer (키워드 기반)
    summary = ""

    if any(k in user_text.lower() for k in ["roommate", "dorm", "noise", "isolate"]):
        summary += "- Roommate conflict impacting emotional well-being.\n"

    if any(k in user_text.lower() for k in ["study", "school", "pressure", "exam", "academic"]):
        summary += "- Strong academic pressure and fear of underperforming.\n"

    if any(k in user_text.lower() for k in ["parents", "family", "expect", "childhood"]):
        summary += "- High parental expectations affecting self-worth.\n"

    if any(k in user_text.lower() for k in ["anxiety", "panic", "sweat", "heartbeat"]):
        summary += "- Anxiety symptoms in social or formal situations.\n"

    if any(k in user_text.lower() for k in ["empty", "lost", "meaning", "value"]):
        summary += "- Feelings of emptiness, loss of direction, existential concerns.\n"

    if any(k in user_text.lower() for k in ["lonely", "alone", "isolated"]):
        summary += "- Persistent loneliness and lack of emotional connection.\n"

    if summary == "":
        summary = "- General emotional distress and interpersonal challenges.\n"

    summaries.append((i, summary.strip()))

# Print the summaries
for idx, summ in summaries:
    print(f"\n===== SUMMARY FOR SAMPLE {idx} =====\n{summ}\n")

In [ ]:
!pwd

### HuggingFace Upload

In [ ]:
!pip install -q "huggingface_hub[cli]"
!huggingface-cli login

In [ ]:
import os
from huggingface_hub import HfApi

api = HfApi(token=os.getenv("  "))

# Original path to the file
file_path = "/content/drive/MyDrive/ProfChuah/PsyDial-en-dialogues-clean.txt"

# Upload the single file using upload_file
api.upload_file(
    path_or_fileobj=file_path,
    path_in_repo="PsyDial-D4-en-dialogues-clean.txt", # Desired filename in the repo
    repo_id="shinnew/PsyDial4_Eng_Translated",
    repo_type="dataset",
    commit_message="Upload PsyDial-D4-en-dialogues-clean.txt"
)

print(f"File {file_path} uploaded successfully to shinnew/PsyDial4_Eng_Translated as PsyDial-D4-en-dialogues-ALL.txt")